In [ ]:
!pip install --upgrade sagemaker datasets

In [ ]:
model_id, model_version = "meta-textgeneration-llama-2-7b", "2.*"

In [ ]:
from sagemaker.jumpstart.estimator import JumpStartEstimator
import boto3

estimator = JumpStartEstimator(model_id=model_id,  environment={"accept_eula": "true"},instance_type = "ml.g5.2xlarge") 

estimator.set_hyperparameters(instruction_tuned="False", epoch="5")

#Fill in the code below with the dataset you want to use from above 
estimator.fit({"training": f"s3://genaiwithawsproject2024/training-datasets/finance"})

In [ ]:
finetuned_predictor = estimator.deploy(instance_type="ml.g5.2xlarge", initial_instance_count=1)

In [ ]:
def print_response(payload, response):
    print(payload["inputs"])
    print(f"> {response}")
    print("\n==================================\n")

In [ ]:
payload = {
    "inputs": "the relative volume for the long out of the money options, indicates",
    "parameters": {
        "max_new_tokens": 64,
        "top_p": 0.9,
        "temperature": 0.6,
        "return_full_text": False,
    },
}
try:
    response = finetuned_predictor.predict(payload, custom_attributes="accept_eula=true")
    print_response(payload, response)
except Exception as e:
    print(e)

In [ ]:
finetuned_predictor.delete_model()
finetuned_predictor.delete_endpoint()